### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [476]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [477]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [478]:
school = school_data["school_name"].count()
student = school_data_complete["student_name"].count()
budget = school_data["budget"].sum()
read = student_data["reading_score"].mean()
math = student_data["math_score"].mean()
pas_math = student_data.loc[student_data["math_score"] >= 70, :].count()
pas_read = student_data.loc[student_data["reading_score"] >= 70, :].count()
pas_all = student_data.loc[(student_data["math_score"] >= 70) & (student_data["reading_score"] >= 70), :].count()
percentage_pas_math = pas_math / student * 100
percentage_pas_read = pas_read / student * 100
percentage_pas_all = pas_all / student * 100
district_df = pd.DataFrame({"Total number of schools": school,"Total number of students": student,"Total budget": budget,
                            "Average math score": math,"Average reading score": read,
                            "Percentage of student pass math": percentage_pas_math,
                            "Percentage of student pass reading": percentage_pas_read,
                            "Percentage of student pass both math and read": percentage_pas_all})
district_df["Total budget"] = district_df["Total budget"].map('${:,.2f}'.format)
district_df["Average reading score"] = district_df["Average reading score"].map('{:,.2f}'.format)
district_df["Average math score"] = district_df["Average math score"].map('{:,.2f}'.format)
district_df["Percentage of student pass math"] = district_df["Percentage of student pass math"].map('{:,.2f}%'.format)
district_df["Percentage of student pass reading"] = district_df["Percentage of student pass reading"].map('{:,.2f}%'.format)
district_df["Percentage of student pass both math and read"] = district_df["Percentage of student pass both math and read"].map('{:,.2f}%'.format)
district_df.rename(index={'Student ID':'District Summary'}, inplace=True)
district_df = district_df.drop(district_df.index[1:11])
district_df

,Total number of schools,Total number of students,Total budget,Average math score,Average reading score,Percentage of student pass math,Percentage of student pass reading,Percentage of student pass both math and read
District Summary,15,39170,"$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [585]:
name_school = school_data["school_name"]
type_school = school_data["type"]
total_student = school_data["size"]
budget_school = school_data["budget"]
#budget_school = school_data["budget"].astype(float).map("${:,.2f}".format)
budget_student = school_data["budget"]/school_data["size"]
#budget_student = budget_student.astype(float).map("${:,.2f}".format)
school_summary_df = pd.DataFrame({"School name": name_school, "School type": type_school, "Total student": total_student,
                                   "Total school budget": budget_school, "Per student budget": budget_student})

In [586]:
summary_df = school_summary_df.sort_values(["School name"]).set_index(["School name"])
per_school = school_data_complete.groupby(["school_name"])
avg_math = per_school["math_score"].mean()
avg_read = per_school["reading_score"].mean()
pass_math = student_data.loc[student_data["math_score"] >= 70, :]
pass_read = student_data.loc[student_data["reading_score"] >= 70, :]
pass_all = student_data.loc[(student_data["math_score"] >= 70) & (student_data["reading_score"] >= 70), :]
school_pass_math = pass_math.groupby(["school_name"])
school_pass_read = pass_read.groupby(["school_name"])
school_pass_all = pass_all.groupby(["school_name"])

In [587]:
per_math = (school_pass_math["school_name"].count()/summary_df["Total student"]) * 100
per_read = (school_pass_read["school_name"].count()/summary_df["Total student"]) * 100
per_all = (school_pass_all["school_name"].count()/summary_df["Total student"]) * 100
summary_df = summary_df.assign(am=avg_math, ar=avg_read, pm=per_math, pr=per_read, pa=per_all)
summary_df = summary_df.rename(columns={"am":"Average math score", "ar":"Average reading score", "pm":"% passing math", 
                                          "pr":"% passing reading", "pa":"% overall passing"})

In [588]:
summary_df

,School type,Total student,Total school budget,Per student budget,Average math score,Average reading score,% passing math,% passing reading,% overall passing
School name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [509]:
top_all = summary_df.sort_values(["% overall passing"], ascending = False)
top_all.head()

,School type,Total student,Total school budget,Per student budget,Average math score,Average reading score,% passing math,% passing reading,% overall passing
School name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [511]:
top_all = summary_df.sort_values(["% overall passing"])
top_all.head()

,School type,Total student,Total school budget,Per student budget,Average math score,Average reading score,% passing math,% passing reading,% overall passing
School name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [525]:
nine = school_data_complete.loc[school_data_complete["grade"] == "9th"]
ten = school_data_complete.loc[school_data_complete["grade"] == "10th"]
ele = school_data_complete.loc[school_data_complete["grade"] == "11th"]
twe = school_data_complete.loc[school_data_complete["grade"] == "12th"]
gp_nine = nine.groupby(["school_name"])
gp_ten = ten.groupby(["school_name"])
gp_ele = ele.groupby(["school_name"])
gp_twe = twe.groupby(["school_name"])

In [526]:
math_nine = gp_nine["math_score"].mean()
math_ten = gp_ten["math_score"].mean()
math_ele = gp_ele["math_score"].mean()
math_twe = gp_twe["math_score"].mean()
msg_df = pd.DataFrame({"9th": math_nine, "10th": math_ten, "11th": math_ele, "12th": math_twe})
msg_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [527]:
read_nine = gp_nine["reading_score"].mean()
read_ten = gp_ten["reading_score"].mean()
read_ele = gp_ele["reading_score"].mean()
read_twe = gp_twe["reading_score"].mean()
rsg_df = pd.DataFrame({"9th": math_nine, "10th": math_ten, "11th": math_ele, "12th": math_twe})
rsg_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [589]:
summary_df.reset_index(drop=False)

,School name,School type,Total student,Total school budget,Per student budget,Average math score,Average reading score,% passing math,% passing reading,% overall passing
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [643]:
bins = [0, 585, 630, 645, 680]
group_labels = ["<$585", "$585-630", "$630-645", "$645-680"]
school_spending_df = summary_df.reset_index(drop=False)
school_spending_df["Spending Range(Per Student)"] = pd.cut(school_spending_df["Per student budget"], bins, labels=group_labels)
spend_stu = school_spending_df.sort_values("Per student budget")
spend_stu = spend_stu[["Spending Range(Per Student)","Average math score","Average reading score",
                       "% passing math", "% passing reading", "% overall passing"]].reset_index(drop=True)
spend_stu.groupby(["Spending Range(Per Student)"]).mean()

,Average math score,Average reading score,% passing math,% passing reading,% overall passing
Spending Range(Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [719]:
summary_df.reset_index(drop=False)

,School name,School type,Total student,Total school budget,Per student budget,Average math score,Average reading score,% passing math,% passing reading,% overall passing
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [722]:
bins_size = [0, 1000, 2000, 5000]
size_labels = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]
school_size_df = summary_df.reset_index(drop=False)
school_size_df["School Size"] = pd.cut(school_size_df["Total student"], bins_size, labels=size_labels)
size_stu = school_size_df.sort_values("Total student")
size_stu = size_stu[["School Size","Average math score","Average reading score",
                       "% passing math", "% passing reading", "% overall passing"]].reset_index(drop=True)
size_stu.groupby(["School Size"]).mean()

,Average math score,Average reading score,% passing math,% passing reading,% overall passing
School Size,,,,,
Small(<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium(1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large(2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [730]:
type_school_df = summary_df.reset_index(drop=False)
type_school_df = type_school_df[["School type", "Average math score",
                                 "Average reading score", "% passing math", "% passing reading", "% overall passing"]]
type_school_df = type_school_df.groupby(["School type"]).mean()
type_school_df

,Average math score,Average reading score,% passing math,% passing reading,% overall passing
School type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
